In [1]:
import torch
import torch.nn.functional as F

def preds2label(pred_same, img1):
    """to detect the img0 label
    
    Arguments:
        pred_same {tensor bs * 1} -- same_label of predict
        img1 {tensor bs * 1} -- img1_label 训练集中用于val进行对比的样本
    
    Returns:
        tensor bs*1 -- predict DX
    """   
    img1_inverse = torch.where((img1!=0), torch.full_like(img1,0) , torch.full_like(img1,1))
    predict_labels = torch.where((pred_same==1),img1, img1_reverse)
    return predict_labels

In [61]:
same =      torch.tensor([[1],[0],[1],[0],[1],[0],[1],[1],[0],[1]])
img1 =      torch.tensor([[0],[1],[3],[0],[3],[0],[1],[3],[0],[1]])
true_img0 = torch.tensor([[3],[0],[1],[1],[0],[1],[3],[3],[1],[1]]).view(-1,)
pred_img0 = predict_label(same, img1).view(-1,)
print(true_img0)
print(pred_img0)
print(true_img0 == pred_img0)
print(torch.sum(true_img0 == pred_img0))
if torch.sum(true_img0 == pred_img0)>5:
    print('ok')

tensor([3, 0, 1, 1, 0, 1, 3, 3, 1, 1])
tensor([0, 0, 3, 1, 3, 1, 1, 3, 1, 1])
tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 1], dtype=torch.uint8)
tensor(6)
ok


In [76]:
o1 = torch.randn(10,128)
o2 = torch.randn(10,128)
label = torch.tensor([1,0,1,1,1,1,0,1,0,0])
print(F.pairwise_distance(o1, o2))
res=torch.abs(o1 - o2)
print(res.shape)
label=label.tolist()
print(label)
result=torch.max(res,1)
print(result)

tensor([15.1433, 16.1523, 15.6637, 15.2236, 16.8706, 16.1007, 15.3622, 16.4850,
        16.3121, 15.4365])
torch.Size([10, 128])
[1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
torch.return_types.max(
values=tensor([3.8407, 4.1431, 4.7672, 4.3962, 4.1973, 3.5278, 3.9685, 4.2204, 3.6245,
        4.2804]),
indices=tensor([30, 99, 13, 84, 62, 95, 16, 44, 53,  5]))


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

# MedicalNet-Tencent 
# https://github.com/Tencent/MedicalNet/blob/master/models/resnet.py
# https://github.com/Tencent/MedicalNet/blob/master/model.py

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]


def conv3x3x3(in_planes, out_planes, stride=1, dilation=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        dilation=dilation,
        stride=stride,
        padding=dilation,
        bias=False)


def downsample_basic_block(x, planes, stride, no_cuda=False):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if not no_cuda:
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride=stride, dilation=dilation)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes, dilation=dilation)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, dilation=dilation, padding=dilation, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 sample_input_D,
                 sample_input_H,
                 sample_input_W,
                 num_seg_classes,
                 shortcut_type='B',
                 no_cuda = False):
        self.inplanes = 64
        self.no_cuda = no_cuda
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            1,
            64,
            kernel_size=7,
            stride=(2, 2, 2),
            padding=(3, 3, 3),
            bias=False)
            
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=1, dilation=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=1, dilation=4)

        self.conv_seg = nn.Sequential(
                                        nn.ConvTranspose3d(
                                        512 * block.expansion,
                                        32,
                                        2,
                                        stride=2
                                        ),
                                        nn.BatchNorm3d(32),
                                        nn.ReLU(inplace=True),
                                        nn.Conv3d(
                                        32,
                                        32,
                                        kernel_size=3,
                                        stride=(1, 1, 1),
                                        padding=(1, 1, 1),
                                        bias=False), 
                                        nn.BatchNorm3d(32),
                                        nn.ReLU(inplace=True),
                                        nn.Conv3d(
                                        32,
                                        num_seg_classes,
                                        kernel_size=1,
                                        stride=(1, 1, 1),
                                        bias=False) 
                                        )

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride,
                    no_cuda=self.no_cuda)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride=stride, dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))

        return nn.Sequential(*layers)

    def forward_once(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # x = self.conv_seg(x)
        
        x = x.view(x.size(0), -1)

        return x

    def forward(self,x1,x2):
        output1 = self.forward_once(x1)
        output2 = self.forward_once(x2)

        return output1,output2


def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

In [4]:
model = resnet18(sample_input_D=49,
                    sample_input_H = 59,
                    sample_input_W = 47,
                    num_seg_classes=2)
model_dict = model.state_dict()
# print(model_dict)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:177: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


In [6]:
pretrained_dict = torch.load('./deep/base_siam/weights/MedicalNet/pretrain/resnet_18.pth')
model_dict = model.state_dict()
print(model_dict)

., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])), ('conv_seg.1.running_var', tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])), ('conv_seg.1.num_batches_tracked', tensor(0)), ('conv_seg.3.weight', tensor([[[[[ 4.7781e-02,  2.2600e-02,  3.1719e-02],
           [ 4.6470e-02,  5.9064e-02,  9.0119e-02],
           [-9.8603e-03,  4.5220e-02, -1.8025e-02]],

          [[-2.9996e-02, -7.5031e-02,  3.6527e-02],
           [ 5.3235e-02,  6.7657e-02,  7.4703e-02],
           [ 5.4255e-03, -1.9369e-02, -4.7950e-02]],

          [[ 6.0310e-03, -5.0718e-03,  8.9277e-02],
           [ 1.9587e-02,  1.3265e-03, -4.4698e-02],
           [ 3.5071e-02, -7.3248e-02,  5.7715e-03]]],


         [[[-2.8861e-03, -4.3450e-02, -2.8001e-02],
           [ 1.0777e-01,  9.9036e-03,  3.0317e-02],
           [ 3.2657e-03, -4.4497e-02, -3.7148e-02]],

          [[-1.1551e-02, -5.0359e-02,  3.7234e-02